# This Notebook converts data to lcdata format

basically the way you should provide the data in input to generate the prediction is a table in lcdata  format, which is explained here https://lcdata.readthedocs.io/en/latest/usage.html (edited)
[2:16 PM] There is a metadata table in it which needs the following keys
- object_id: A unique identifer. Default: randomly assigned string
- ra: The right ascension. Default: nan
- dec: The declination. Default: nan
- type: A string representing the type of the light curve. Default: Unknown
- redshift: The redshift. Default: nan
(edited)

[2:16 PM] And a lightcurve table with
- time: times at which the light curve was sampled. Converted to a 64-bit float.
- flux: The flux at each point on the light curve. Converted to a 32-bit float.
- fluxerr: The uncertainty on the flux. Converted to a 32-bit float.
- band: A string representing bandpass that the light curve was observed in. We recommend using the sncosmo bandpass names here. Converted to a binary string.

In [15]:
import sys
# For importing useful functions
#ls: cannot access '/global/homes/p/portmanm/timedomain/gwtarget/': Permission denied
sys.path.append('/global/homes/p/portmanm/timedomain/gwtarget/')

from astropy.io import fits
from astropy.table import Table, Column, join, hstack, vstack, unique, setdiff
from astropy import units as u
from astropy.coordinates import SkyCoord, match_coordinates_sky, Angle
from astropy.time import Time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import sys
import lcdata

import sqlite3
import os
import h5py
import json

In [2]:
data = pd.read_csv('LCData_Legacy/A202103221407558m001825/lc_A202103221407558m001825.csv', delimiter=',')

In [3]:
data['filter'] = data['filter'].replace(['r','g','z'],['desr','desg', 'desz'])
data.rename(columns = {'filter':'band'}, inplace = True)
alert_mask=data['alert']
masked_data=data[alert_mask]
masked_data

,mjd,band,mag,magerr,lm5,sciname,alert
1,59295.277999,desr,22.158561,0.066419,24.040510,c4d_210322_064019_ooi_r_v1.fits.fz,True
3,59297.231663,desg,22.913798,0.085523,24.528934,c4d_210324_053335_ooi_g_v1.fits.fz,True
4,59297.232705,desr,22.226883,0.085047,23.878723,c4d_210324_053505_ooi_r_v1.fits.fz,True
16,59310.269789,desr,22.257689,0.099513,23.693262,c4d_210406_062829_ooi_r_v1.fits.fz,True
17,59310.270937,desz,22.009473,0.147323,22.989297,c4d_210406_063008_ooi_z_v1.fits.fz,True
20,59313.222603,desr,22.194041,0.069099,24.050969,c4d_210409_052032_ooi_r_v1.fits.fz,True
21,59313.223773,desz,22.141966,0.120006,23.349886,c4d_210409_052214_ooi_z_v1.fits.fz,True
24,59316.208513,desr,22.650199,0.111777,23.965790,c4d_210412_050015_ooi_r_v1.fits.fz,True
25,59316.209662,desz,22.068240,0.142409,23.098177,c4d_210412_050154_ooi_z_v1.fits.fz,True
28,59319.245280,desr,22.531287,0.093837,24.025299,c4d_210415_055312_ooi_r_v1.fits.fz,True


In [17]:
masked_data = masked_data.drop(columns='alert')

## The MJD for a given date is the number of days to that date since Jan 1 4713 B.C. 00:00:00 (midnight). By definition, MJD includes a time component expressed as a decimal, which represents some fraction of 24 hours

In [18]:
masked_data.rename(columns = {'mjd':'time'}, inplace = True)
masked_data

,time,band,mag,magerr,lm5,sciname
1,59295.277999,desr,22.158561,0.066419,24.040510,c4d_210322_064019_ooi_r_v1.fits.fz
3,59297.231663,desg,22.913798,0.085523,24.528934,c4d_210324_053335_ooi_g_v1.fits.fz
4,59297.232705,desr,22.226883,0.085047,23.878723,c4d_210324_053505_ooi_r_v1.fits.fz
16,59310.269789,desr,22.257689,0.099513,23.693262,c4d_210406_062829_ooi_r_v1.fits.fz
17,59310.270937,desz,22.009473,0.147323,22.989297,c4d_210406_063008_ooi_z_v1.fits.fz
20,59313.222603,desr,22.194041,0.069099,24.050969,c4d_210409_052032_ooi_r_v1.fits.fz
21,59313.223773,desz,22.141966,0.120006,23.349886,c4d_210409_052214_ooi_z_v1.fits.fz
24,59316.208513,desr,22.650199,0.111777,23.965790,c4d_210412_050015_ooi_r_v1.fits.fz
25,59316.209662,desz,22.068240,0.142409,23.098177,c4d_210412_050154_ooi_z_v1.fits.fz
28,59319.245280,desr,22.531287,0.093837,24.025299,c4d_210415_055312_ooi_r_v1.fits.fz


In [28]:
#dataset = lcdata.from_light_curves([masked_data])

# flux is actually magnitude here
light_curve = Table({
     'time': list(masked_data['time']),
     'flux': list(masked_data['mag']),
     'fluxerr': list(masked_data['magerr']),
     'band': list(masked_data['band']),})

print(light_curve, '\n')


with open('LCData_Legacy/A202103221407558m001825/object-summary.json') as f:
    meta_data = json.load(f)
    light_curve.meta = {
        'object_id': meta_data['ObjectID'],
        'ra': meta_data['RA-OBJECT'],
        'dec': meta_data['DEC-OBJECT'],
        'type': 'Unknown',
        'redshift': float('nan')}
    
    for i in meta_data:
        print(i,':   ', meta_data[i])

        
print('\n', light_curve.meta)

       time               flux             fluxerr       band
------------------ ------------------ ------------------ ----
 59295.27799854438 22.158561218035263 0.0664186639562546 desr
 59297.23166343352 22.913797890107617 0.0855227030264499 desg
 59297.23270480759  22.22688270654593 0.0850471428225507 desr
 59310.26978909521  22.25768900093766 0.0995130268852795 desr
59310.270936665096  22.00947349716405 0.1473225427504925 desz
  59313.2226029866  22.19404078465104 0.0690990376287754 desr
 59313.22377329604 22.141965546645128 0.1200055981924307 desz
 59316.20851268766  22.65019948952747 0.1117767303252109 desr
 59316.20966174384   22.0682400374456 0.1424085059677942 desz
59319.245280151896 22.531287115248617 0.0938369049627861 desr
 59319.24643760768  22.65978726557133 0.2251451149168614 desz
 59322.23467318657  22.82592029552405 0.1241239451055852 desr 

ObjectID :    A202103221407558m001825
RA-OBJECT :    211.98278629
DEC-OBJECT :    -0.30695087
NumberAlerts :    12
MaxSCORE :    0

In [29]:
dataset = lcdata.from_light_curves([light_curve])

In [30]:
dataset.meta

object_id,ra,dec,type,redshift
str23,float64,float64,str7,float64
A202103221407558m001825,211.98278629,-0.30695087,Unknown,nan


In [32]:
dataset.light_curves

array([<Table length=12>
              time           flux     fluxerr    band
            float64        float32    float32   bytes4
       ------------------ --------- ----------- ------
        59295.27799854438 22.158562  0.06641866   desr
        59297.23166343352 22.913797 0.085522704   desg
        59297.23270480759 22.226883  0.08504714   desr
        59310.26978909521 22.257689 0.099513024   desr
       59310.270936665096 22.009474  0.14732254   desz
         59313.2226029866  22.19404  0.06909904   desr
        59313.22377329604 22.141966   0.1200056   desz
        59316.20851268766   22.6502  0.11177673   desr
        59316.20966174384  22.06824   0.1424085   desz
       59319.245280151896 22.531286   0.0938369   desr
        59319.24643760768 22.659788  0.22514512   desz
        59322.23467318657  22.82592 0.124123946   desr], dtype=object)

In [ ]:
files = {}
listdir = ['/global/project/projectdirs/cosmo/data/legacysurvey/dr9/north/sweep/9.0', '/global/project/projectdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0']

for dirname, dirnames, filenames in os.walk(listdir[1]):
    # print path to all subdirectories first.
    for subdirname in dirnames:
        files.append(os.path.join(dirname, subdirname))

    # print path to all filenames.
    for filename in filenames:
        block = filename[6:-5]
        
        if block[6].isdigit():
            #print(block)
            min_ra = int(block[:3])
            min_dec = int(block[4:7])
            if block[3] == 'm':
                min_dec = int('-'+str(min_dec))
                
            max_ra = int(block[8:11])
            max_dec = int(block[12:])
            if block[-4] == 'm':
                max_dec = int('-'+str(max_dec))
                
            #print('minra:',min_ra, 'mindec', min_dec, '\nmaxra',max_ra, 'maxdec', max_dec)
            files[filename]={'min_ra':min_ra, 'max_ra':max_ra, 'min_dec': min_dec, 'max_dec':max_dec, 'path':os.path.join(dirname, filename)}


path = find_host_galaxy_file(ra, dec)
print(path)

print(ra, dec)
def find_host_galaxy_file(ra, dec):
    for i in files.keys():
         if ra>=files[i]['min_ra'] and ra<=files[i]['max_ra'] and dec>=files[i]['min_dec'] and dec<=files[i]['max_dec']:
            return files[i]['path']
         else: continue
    return float(np.NaN)

path = find_host_galaxy_file(ra, dec)
print(path)

def find_redshift(path):
    sweep_pz = path.replace('9.0', '9.0-photo-z').replace('.fits', '-pz.fits')
        print(sweep_pz)
        if os.path.exists(sweep_pz):
            fits.open(sweep_pz)
            
        else:
            print(os.path.exists(sweep_pz))
            return float(np.NaN)
        

In [ ]:
# '/global/project/projectdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0' files for the host galaxies which include redshift.
# pip instal --user
# get spectroscopic redshift


def get_objects_in_gw_contour(sweepfolders, lvc_hp_file, level=0.9):
    """Return a list of Legacy Survey objects inside a GW contour.

    Parameters
    ----------
    sweepfolders : str or list
        A folder or list of folders to search for sweep files.
    lvc_hp_file : str
        HEALPix file with LVC event reconstruction.
    level : float
        Contour level, in [0..1].

    Returns
    -------
    objtab : astropy.Table
        Table of Legacy Survey objects inside GW contour.
    """
    # Get luminosity distance to GW marginalized over all angles.
 #find my oun pixid using lightcurve dec, ra

    pidxids = ...

    # Loop through all sweeps, see if pixel indices match with any bricks.
    sweepfiles = get_sweepfiles(sweepfolders)
    gwsweeps = []
    for sf in sweepfiles:
        ramin, ramax, decmin, decmax = io.decode_sweep_name(sf)
        ra = [ramin, ramax, ramax, ramin]
        dec = [decmin, decmin, decmax, decmax]
        box = hp.ang2vec(ra, dec, lonlat=True)
        boxpix = hp.query_polygon(nside, box, nest=True)

        if not set(boxpix).isdisjoint(pixids):
            gwsweeps.append(sf)

    # Grab photo-z data and read everything into a table.
    gwspeeps, photozs = get_sweep_photo_z(gwsweeps)

    pixarea_deg2 = hp.nside2pixarea(nside, degrees=True)

    objtab = None
    for sf, pz in zip(gwsweeps, photozs):
        print(sf)
        data = Table(io.read_tractor(sf, columns=['BRICKID', 'BRICKNAME', 'OBJID', 'TYPE', 'RA', 'DEC',
                                                  'FLUX_G', 'FLUX_R', 'FLUX_Z',
                                                  'NOBS_G', 'NOBS_R', 'NOBS_Z',
                                                  'FRACIN_G', 'FRACIN_R', 'FRACIN_Z',
                                                  'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z',
                                                  'PMRA', 'PMDEC', 'MASKBITS'
                                                  ]))
        data_pz = Table.read(pz)

        # Cut pixels not in 90% region.
        brickpix = hp.ang2pix(nside, data['RA'], data['DEC'], lonlat=True, nest=True)
        brickringpix = hp.nest2ring(nside, brickpix)



        if objtab is None:
            objtab = hstack([data, data_pz[select]])
        else:
            objtab = vstack([objtab, hstack([data[select], data_pz[select]])])

    # Compute luminosity distance given mean photo-z.
#     nonzero_prob = objtab['angprob'] > 0
#     http://logging.info('Computing luminosity distances for {:d} objects.'.format(np.sum(nonzero_prob)))

#     d_L = np.zeros(len(objtab))
#     nonneg_z = objtab['Z_PHOT_MEAN'] >= 0
#     d_L[nonzero_prob & nonneg_z] = cosmo.luminosity_distance(objtab[nonzero_prob & nonneg_z]['z_phot_mean']).to('Mpc').value
#     objtab.add_column(Column(name='D_L', data=d_L, unit='Mpc'))

    return objtab



In [ ]:
def find_redshift(path, ra, dec):
    '''Finds Spectroscopic and Photometric redshift and Standard Deviation
       NOTE: - Spectroscopic Redshift will be -99.0 if it does not exist
             - I the associated photo-z file does not exist, returns a tuple
               of type nan
       
    Parameters
    ----------
    sweepfile_path : str
        A sweep file path associated with the matching brick.
    
    Returns
    -------
     z_spec, z_photo_mean, z_photo_err: tuple
        A tuple containing spectroscopic redshift, mean of photometric redshift,
        and photometric redshift standard deviation respectively of the matching
        object within the sweep file.
    '''
    c = SkyCoord(ra*u.degree, dec*u.degree)
    catalog = SkyCoord(a[1]*u.degree, a[2]*u.degree)
    idx, d2d, d3d = c.match_to_catalog_3d(catalog)
    print(idx, d2d, d3d)
    obj_id = a[0][idx]
    
    catalog_match = catalog[idx]
    print(catalog_match, type(catalog_match))
    ra_match = catalog_match.ra*u.degree
    dec_match = catalog_match.dec*u.degree

    print('objid:', obj_id)
    
    sweep_pz = path.replace('9.0', '9.0-photo-z').replace('.fits', '-pz.fits')
    print(sweep_pz)
    if os.path.exists(sweep_pz):
        hdus = fits.open(sweep_pz)
        print(hdus.info())
        header = hdus[1].header
        redshift_data = Table(hdus[1].data)
        assert redshift_data['OBJID'][idx] == obj_id 
        print(list(redshift_data.keys()))
        print(redshift_data['Z_SPEC'][idx])
        z_spec, z_photo_mean, z_photo_err = (redshift_data['Z_SPEC'][idx],redshift_data['Z_PHOT_MEAN'][idx], redshift_data['Z_PHOT_STD'][idx])
        return (z_spec, z_photo_mean, z_photo_err)            
    else:
        print(os.path.exists(sweep_pz))
        return (np.NaN, np.NaN, np.NaN)

z_spec, z_photo_mean, z_photo_err = find_redshift(path, ra_pseudo_host, dec_pseudo_host)    
    
"""max_sep = 1.0 * u.arcsec
 idx, d2d, d3d = c.match_to_catalog_3d(catalog)
 sep_constraint = d2d < max_sep
 c_matches = c[sep_constraint]
 catalog_matches = catalog[idx[sep_constraint]]
 
 Use Z_SPEC if available, else use Z_PHOTO_MEAN
 """
print(float(np.NaN))
print('z_spec:', z_spec,'mean:', z_photo_mean,'err:', z_photo_err)